In [1]:
import pandas as pd
import numpy as np
from tqdm.auto import tqdm
tqdm.pandas()

df = pd.read_csv('./data/October_missions_full.csv')
df['mission'] = df['type'] + '_' + df['target'].astype(str)

df = df[['user', 'mission', 'createdAt', 'type', 'target', 'completed', 'performance']]
df['createdAt'] = pd.to_datetime(df['createdAt'], format='mixed').dt.date
df['user'] = df['user'].astype('category').cat.codes
df['mission'] = df['mission'].astype('category')
df['type'] = df['type'].astype('category')

mean_size = df.groupby('user').size().mean()
df = df.groupby('user').filter(lambda x: len(x) > mean_size)

df.sort_values(by=['createdAt'], inplace=True, ignore_index=True)
df

,user,mission,createdAt,type,target,completed,performance
0,3953,quiz_4,2024-10-01,quiz,4,False,0.000000
1,2526,streak_1,2024-10-01,streak,1,True,1.000000
2,1219,activity_5,2024-10-01,activity,5,True,1.000000
3,223,quiz_6,2024-10-01,quiz,6,False,0.166667
4,4263,quiz_3,2024-10-01,quiz,3,False,0.000000
...,...,...,...,...,...,...,...
66063,4859,mobility_1,2024-10-31,mobility,1,False,0.000000
66064,392,mobility_1,2024-10-31,mobility,1,False,0.000000
66065,4906,activity_4,2024-10-31,activity,4,True,1.000000
66066,5332,mobility_1,2024-10-31,mobility,1,False,0.000000


In [ ]:
def get_past_interaction(row, df):
    user = row['user']
    createdAtT = row['createdAt']
    past_interactions: pd.DataFrame = df[(df['user'] == user) & (df['createdAt'] < createdAtT)]
    return pd.Series({
        'user': user,
        'mission': row['mission'],
        'past_interactions': past_interactions[['type', 'target', 'performance']]
    })

seq_df = df.progress_apply(get_past_interaction, axis=1, args=(df,))
seq_df

  0%|          | 0/15 [00:00<?, ?it/s]

,user,mission,past_interactions
66053,3147,activity_8,type target performance 429 ac...
66054,6861,streak_1,type target performance 301 mo...
66055,3294,quiz_6,type target performance 21985 ...
66056,3708,mobility_2,type target performance 46128 ...
66057,519,mobility_1,type target performance 13902 ac...
66058,4408,episode_3,type target performance 661 ...
66059,6884,episode_3,type target performance 374 ...
66060,6241,streak_1,type target performance 31095 ...
66061,3896,activity_1,type target performance 1341 ...
66062,6942,episode_1,type target performance 37556 mo...


In [3]:
x: pd.DataFrame = seq_df.loc[66066, 'past_interactions']

x

,type,target,performance
553,quiz,6,0.0
1286,action,1,1.0
1475,episode,4,1.0
1682,mobility,2,1.0
1983,episode,2,1.0
...,...,...,...
59386,quiz,6,1.0
60921,activity,2,1.0
61867,episode,2,1.0
62637,activity,3,1.0
